In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import warnings
warnings.filterwarnings('ignore')
from Preprocessing import date_splitter,time_trier,route,tod_departure

In [14]:
trainset = pd.read_csv(r'../Data/trainset.csv')
testset = pd.read_csv(r'../Data/testset.csv')

In [15]:
X_train, y_train = trainset.iloc[:,:-1], trainset.iloc[:,-1]
X_test, y_test = testset.iloc[:,:-1], testset.iloc[:,-1]